# Handle images using Bio-Formats
Java is a popular programming language and image processing software like [ImageJ](https://imagej.net/ij/) or [Fiji](https://imagej.net/software/fiji/) have been used by many scientists for decades.

In order to analyse your image data, you need to be able to read the numerous proprietary file formats (PFF).
[Bio-Formats](
https://www.openmicroscopy.org/bio-formats/) is the most comprehensive software tool for reading and writing image data using standardized, open formats. Bio-Formats is a community driven Java project with a standardized application interface that supports open source analysis programs like ImageJ, Fiji.
Bio-Formats is a library of readers than can easily be extended. This is outside the scope of this course.

To explore the library, you can use images [Sample Images](https://downloads.openmicroscopy.org/images/). Alternatively, you can use your own images, or public repositories such as [Zenodo](https://zenodo.org) or [IDR](https://idr.openmicroscopy.org/about/download.html).


## Learning Objectives

* How to use Bio-Formats for reading images
* How to use Bio-Formats for converting images

## Installation

Bio-Formats being a Java library, the first step is to install Java.
You can either install it globally (see instructions) or in a virtual environment e.g. Conda environment.

### Install Bio-Formats Command Line Tools

1. Download [Bio-Formats Command Line Tools (bftools)](https://downloads.openmicroscopy.org/bio-formats/8.0.1/artifacts/bftools.zip).
2. Find the downloaded `bftools.zip` (most probably in the ``Downloads`` folder) and unzip it.
3. The tools are portable, i.e. you can run them from the directory you unzip them to and they don't need an installer.
4. Open a terminal (Command prompt) and go into the new `bftools` folder using the ``cd`` command
   ```
   cd /PATH/TO/BFTOOLS/bftools
   ```

## Open image using Bio-Formats Command Line

The [Command line tools overview](https://bio-formats.readthedocs.io/en/stable/users/comlinetools/index.html) describes all the commands available.

The following command lists the supported formats.

In [ ]:
formatlist

To check if a specific format is supported, us the ``grep`` command with the file extension.
Let's check, for example, the ``.lif`` extension. Two readers should be returned

In [ ]:
formatlist | grep lif
Leica Image File Format: can read (lif)
Openlab LIFF: can read (liff)

### Reading use case
Let's download a Leica file (`.lif`) for example from [imagesc-30856](https://downloads.openmicroscopy.org/images/Leica-LIF/imagesc-30856/) and read it. Note that the file name contains whitespaces.

Whitespaces in file and folder names are problematic when working on the command line. If you try and pass a file name that contains spaces as an option to a command line program, the command interpreter sees several separate arguments. You can either rename the files or use quotes around each element to tell the command line interpreter that the whitespaces are part of the file name.

The ``showinf`` command will open the image in a viewer. It will also show the image info as an output on the command line.
User ``Ctrl+C`` to get back to the command prompt (the image will close). 

```
showinf /PATH/TO/DOWNLOADED/IMAGE
Reading core metadata
filename = /PATH/TO/DOWNLOADED/IMAGE
Series count = 8
Series #0 :
	Image count = 2
	RGB = false (1) 
	Interleaved = false
	Indexed = true (false color, 8-bit LUT: 3 x 256)
	Width = 1024
	Height = 1024
	SizeZ = 1
	SizeT = 1
	SizeC = 2
	Tile size = 1024 x 1024
	Thumbnail size = 128 x 128
	Endianness = intel (little)
	Dimension order = XYCZT (certain)
	Pixel type = uint8
	Valid bits per pixel = 8
	Metadata complete = true
	Thumbnail series = false
	-----
	Plane #0 <=> Z 0, C 0, T 0
	Plane #1 <=> Z 0, C 1, T 0

Series #1 :
	Image count = 2
	RGB = false (1) 
	Interleaved = false
	Indexed = true (false color, 8-bit LUT: 3 x 256)
	Width = 1024
...
```

## Bio-Formats as a conversion tool
Often, rather than just reading an image and resaving it in a different format, it also makes sense to split the images into smaller parts, for example one might want to split a single file that contains images from a multiple-position scan into multiple individual files, where each file contains only images from a single position. Similarly, one may want to split images between different channels, time points or Z-slices. To keep track of the particular position (e.g. the well of a multiwell plate) or time point/channel/Z-slice that an image came from in the subsequent batch analysis workflow, it can be useful to embed such metadata in the filename.

Very often the conversion is done into ``.tif`` format.

The approach of splitting, converting and renaming can have some benefits:

- It allows for standardization in an image analysis workflow even in a heterogeneous facility environment with many different microscopes.
- By using the extra step of renaming/conversion, the same pipeline/workflow can be used for images originating from different microscope systems.
- Dealing with single large files often has performance penalties, handling smaller files can be more efficient.
- Analysing or sharing a subset of the data is more easily done on a collection of files (this can simply be done in any file manager) rather than with a single file which can exceed hundreds of Gigabytes for a large time lapse experiment.

### Common use case

Often you will use ``bfconvert`` in the following way:

```
bfconvert inputfile  outputfilepattern
```
or 
```
bfconvert -padded inputfile  outputfilepattern
```

where ``inputfile`` is the path to the input file and ``outputfilepattern`` specifies the output file path and format string.

The ``outputfilepattern`` is not a single filename but rather a format string that defines a filename template. After all we want to split a single file into many files, so we need to specify how the many files are named. The command ``bfconvert`` replaces parts of the format string with the index number or name of the particular subset of the input file that is written to a given output file, according to the following pattern:

```
   Pattern:     Metadata value:
   ---------------------------
   %s           series index
   %n           series name
   %c           channel index
   %w           channel name
   %z           Z index
   %t           T index
   %A           acquisition timestamp
   %x           row index of the tile
   %y           column index of the tile
   %m           overall tile index
```

For a given file format you may have to try out first what series and other metadata fields correspond to.

For example, for ``.lif`` files (Leica), the different series indices correspond to different positions of a multiposition scan.

``-padded`` zero-padding of numbers

Very often, you will want to pass the ``-padded`` option. This makes sure that all numbers in the filename will have the same number of digits, e.g. ``1`` will be converted to ``001`` if there are more than 100 images in the series. This helps with sorting files in tools useing alphabetical sorting.

### Converting to OME-TIFF


The [OME-TIFF](https://ome-model.readthedocs.io/en/stable/ome-tiff/) format was created to maximize the respective strengths of [OME-XML](https://ome-model.readthedocs.io/en/stable/ome-xml/), a file format for storing microscopy information (both pixels and metadata) using the [OME Data Model](https://ome-model.readthedocs.io/en/stable/) and [TIFF](https://en.wikipedia.org/wiki/TIFF). It takes advantage of the rich metadata defined in OME-XML while retaining the pixels in multi-page TIFF format for compatibility with many more applications.

An OME-TIFF dataset has the following characteristics:

- Image planes are stored within one multi-page TIFF file, or across multiple TIFF files. Any image organization is feasible.
- A complete OME-XML metadata block describing the dataset is embedded in each TIFF file’s header. Thus, even if some of the TIFF files in a dataset are misplaced, the metadata remains intact.
- The OME-XML metadata block may contain anything allowed in a standard OME-XML file.
- OME-TIFF uses the standard TIFF mechanism for storing one or more image planes in each of the constituent files, instead of encoding pixels as base64 chunks within the XML. Since TIFF is an image format, it makes sense to only use OME-TIFF as opposed to OME-XML, when there is at least one image plane.

Converting to OME-TIFF is a common mechanism used to open files in Software not supporting the original format, including proprietary Software e.g. [Imaris](https://imaris.oxinst.com/)

Let's now convert the ``.lif`` file into OME-TIFF (``.ome.tiff``) Either use quotes or replace, for example, whitespaces by underscore.
```
bfconvert "20191025 Test FRET 585. 423, 426.lif" output.ome.tiff
```

Read the metadata only

```
showinf -nopix output.ome.tiff
```

To convert files in batch, you will often use a ``bash`` script i.e. a Unix command line interface responsible for interacting with a computer's operating system. 
The code snippet below shows how to loop through a directory and convert the ``.lif`` files.
This is the content of ``convert.sh`` file.

```
#!/bin/bash

directory=$1

for file in "$directory"/*.lif; do
    # check that it is a file
    if [ -f "$file" ]; then
      # print the file
      echo "$file"
      filename=$(basename -- "$file")
      extension="${filename##*.}"
      filename="${filename%.*}" # get the name of the file
      bfconvert "$file" "$filename".ome.tiff
    fi
  done

```
The script assumes that it is placed in the same directory where ``bfconvert`` can be executed. To run it from a different location, 
replace ``bfconvert`` by ``FULL_PATH_TO/bfconvert``

To execute the script:
1. Open a terminal
2. Run ``bash convert.sh NAME_OF_DIRECTORY``. 

Other command line converters based on Bio-Formats are available to convert to OME-TIFF e.g. [raw2ometiff](https://github.com/glencoesoftware/raw2ometiff), [bioformats2raw](https://github.com/glencoesoftware/bioformats2raw).
Converting using the command line is useful when creating or managing a large volume of data since it is straightforward to write a command line script looping, for example, over all the files in a given folder.

Graphical User Interface tools based on Bio-Formats also exist to convert images into OME-TIFF.
One example is the freely available [NGFF converter](https://www.glencoesoftware.com/products/ngff-converter/) developed by [GlencoeSoftware](https://www.glencoesoftware.com/).

### Converting using the NGFF converter
1. Download from [downloads.glencoesoftware.com](https://downloads.glencoesoftware.com/public/NGFF-Converter/latest/) the executable corresponding to your operating system.
2. Open the converter.
3. Drag and drop the ``.lif`` file into the converter pane.
4. In the next window, select ``OME-TIFF`` as the output format and convert it into ``OME-TIFF`` by clicking the play button under the ``Actions`` tab.

### License (BSD 2-Clause)
Copyright (C) 2025 University of Dundee. All Rights Reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.